In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
dt=pd.read_csv('../input/dta3.csv')


In [ ]:
dt=pd.get_dummies(dt)

In [ ]:
dt=dt.drop(columns=['B_ELECTRONIC_AUCTION_N','PROCEDURE_RES','SECTOR_na','B_GPA_N','TYPE_OF_CONTRACT_W','CAE_TYPE_Z','AUTHORITY_COUNTRY_UK'] )

In [ ]:
print(dt.columns)
len(dt)

In [ ]:
dt = dt[pd.notnull(dt['AWARD_VALUE_EURO'])]
dt = dt[dt['AWARD_VALUE_EURO'] != 0]
len(dt)



In [ ]:
dt = dt[pd.notnull(dt['AWARD_EST_VALUE_EURO'])]
dt = dt[dt['AWARD_EST_VALUE_EURO'] != 0]
len(dt)



In [ ]:
dt = dt[pd.notnull(dt['NUMBER_OFFERS'])]
dt = dt[dt['NUMBER_OFFERS'] != 0]
len(dt)

In [ ]:
import numpy as np
dt['LNRATIO']=(np.log(dt['AWARD_VALUE_EURO']/dt['AWARD_EST_VALUE_EURO']))
len(dt)


In [ ]:
#log of 'df_train['month'] = pd.to_datetime(df_train.Date).dt.month
dt['LN_AWARD_VALUE_EURO'] = np.log(dt['AWARD_VALUE_EURO'])
dt['LN_AWARD_EST_VALUE_EURO'] = np.log(dt['AWARD_EST_VALUE_EURO'])

In [ ]:
dt['LNRATIO'].hist(bins=100)

In [ ]:
dt['LNRATIO'].describe()

In [ ]:
dt = dt[(dt.LNRATIO>-3) & (dt.LNRATIO<3)]


In [ ]:
dt['LNRATIO'].describe()

In [ ]:
dt['LN_AWARD_VALUE_EURO'].hist(bins=1000)

In [ ]:
dt['LN_AWARD_EST_VALUE_EURO'].hist(bins=1000)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter( dt['LN_AWARD_VALUE_EURO'],dt['LN_AWARD_EST_VALUE_EURO'])

In [ ]:
train=dt[(dt.YEAR<2017)]
test=dt[(dt.YEAR==2017)]

In [ ]:
y_train = train['LNRATIO']
X_train = train.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO', 'LNRATIO', 'LN_AWARD_VALUE_EURO'], axis=1)
y_test = test['LNRATIO']
X_test = test.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO',  'LNRATIO' ,'LN_AWARD_VALUE_EURO'], axis=1)


In [ ]:
X_train.columns

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
mOLS = sm.OLS(y_train, X_train).fit()
print(mOLS.summary())



# Print out the statistics

# Fit and make the predictions by the model


In [ ]:
import math
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    print('Train rmse '+str(rmse(m.predict(X_train), y_train))) 
    print('Test  rmse '+str(rmse(m.predict(X_test), y_test)))
    print('Train '+str(spearmanr(m.predict(X_train),y_train)))
    print('Test  '+str(spearmanr(m.predict(X_test),y_test)))
    

In [ ]:
print_score(mOLS)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, mOLS.predict(X_test))

In [ ]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
lg = lgb.LGBMRegressor(silent=False)
param_dist = {"max_depth": [-1,10, 50],
              "learning_rate" : [0.01,0.1,0.2],
              "num_leaves": [10,100,500],
              "n_estimators": [200,500,1000]
             }
grid_search = GridSearchCV(lg,  param_grid=param_dist, cv = 2,scoring='neg_mean_squared_error' ,verbose=5)
grid_search.fit(X_train,y_train)
print(grid_search.best_estimator_)

In [ ]:
print(grid_search.best_params_)

In [ ]:
import math
from scipy.stats import spearmanr
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import gc
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
folds = KFold(n_splits=4, shuffle=True, random_state=42)
oof_reg_preds = np.zeros(X_train.shape[0])
sub_reg_preds = np.zeros(X_test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    trn_x, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    reg = LGBMRegressor(**grid_search.best_params_)
   
    print("-"* 20 + "LightGBM Training" + "-"* 20)
    reg.fit(trn_x, trn_y, eval_set= [ (val_x, val_y)],early_stopping_rounds=100,verbose=200,eval_metric='rmse' )
    # LightGBM
    oof_reg_preds[val_idx] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    sub_reg_preds  += reg.predict(X_test, num_iteration=reg.best_iteration_)/ folds.n_splits
    print('Fold %2d RMSE: %.6f' % (n_fold + 1, rmse(val_y, oof_reg_preds[val_idx])))
    del reg, trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full Train RMSE score %.6f' % rmse(y_train, oof_reg_preds)) 
print('Full Test RMSE score %.6f' % rmse(y_test, sub_reg_preds))    
print('Train '+str(spearmanr(oof_reg_preds,y_train)))
print('Test  '+str(spearmanr(sub_reg_preds,y_test)))  


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, sub_reg_preds)

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
mod1 = XGBRegressor()
param= {"max_depth": [10,50],
              "min_child_weight" : [1,6],
              "n_estimators": [200,500],
              "learning_rate": [0.05, 0.2]}
search1 = GridSearchCV(mod1, param,scoring='neg_mean_squared_error', cv = 2,verbose=True)
search1.fit(X_train.values,y_train.values)

print(search1.best_params_)

In [ ]:
xgb_params = {
        'objective': 'reg:linear',
        'booster': 'gbtree',
        'learning_rate': 0.1,
        'max_depth': 12,
        'min_child_weight': 300,
        'gamma' : 1.45,
        'alpha': 0.0,
        'lambda': 1.0,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'colsample_bylevel': 1,
        'n_jobs': 1,
        'random_state': 456 
    }
from sklearn.model_selection import KFold
import xgboost as xgb
import gc
import math
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
folds = KFold(n_splits=4, shuffle=True, random_state=42)
oof_reg_preds1 = np.zeros(X_train.shape[0])
xgb_preds= np.zeros(X_test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    trn_x, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    d_train = xgb.DMatrix(trn_x.values, trn_y.values)
    d_valid = xgb.DMatrix(val_x.values, val_y.values)
    d_test = xgb.DMatrix(X_test.values)
    d_val=xgb.DMatrix(val_x.values)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb.XGBRegressor(**search1.best_params_)
    print("-"* 20 + "XGboost Training" + "-"* 20)
    xgb.train(xgb_params, d_train,100, watchlist,verbose_eval=50,early_stopping_rounds=10)
   
   
    
    
    # Xgboost
    oof_reg_preds1[val_idx] = xgb.predict(d_val)
    xgb_preds += xgb.predict(d_test)/ folds.n_splits
                        
   
   
    
    
   
    print('Fold %2d RMSE: %.6f' % (n_fold + 1, rmse(val_y, oof_reg_preds1[val_idx])))
    del  trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full RMSE score %.6f' % rmse(y_train, oof_reg_preds1)) 
print('Full Test RMSE score %.6f' % rmse(y_test, xgb_preds))    
print('Train '+str(spearmanr(oof_reg_preds1,y_train)))
print('Test  '+str(spearmanr(xgb_preds,y_test)))  


In [ ]:
import catboost as cb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
params = {'depth': [5,8,10],
          'learning_rate' : [ 0.01,0.1,0.2],    
         'iterations': [200,500,1000]}
cb = cb.CatBoostRegressor()
cb_model = GridSearchCV(cb, params, scoring='neg_mean_squared_error', cv = 2,verbose=False,n_jobs=-1)
cb_model.fit(X_train, y_train)
print(cb_model.best_params_)

In [ ]:
print(cb_model.best_params_)

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
import gc
import math
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
folds = KFold(n_splits=4, shuffle=True, random_state=42)
oof_reg_preds2 = np.zeros(X_train.shape[0])
cat_preds= np.zeros(X_test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    trn_x, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    cat = CatBoostRegressor(**cb_model.best_params_)
   
    print("-"* 20 + "Catboost Training" + "-"* 20)
    cat.fit(trn_x, trn_y,eval_set= [ (val_x, val_y)],early_stopping_rounds=50,use_best_model=True,verbose=100)
    
    
    # catboost
    oof_reg_preds2[val_idx] = cat.predict(val_x)
    cat_preds +=cat.predict(X_test)/ folds.n_splits
    
    
    
   
    print('Fold %2d RMSE: %.6f' % (n_fold + 1, rmse(val_y, oof_reg_preds2[val_idx])))
    del  trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full Train RMSE score %.6f' % rmse(y_train, oof_reg_preds2)) 
print('Full Test RMSE score %.6f' % rmse(y_test, cat_preds))    
print('Train '+str(spearmanr(oof_reg_preds2,y_train)))
print('Test  '+str(spearmanr(cat_preds,y_test)))